# Optimization pipeline

## QUBO Marix

We have a square matrix W

In [34]:
import numpy as np
W = [[153, 510, 706, 966, 581, 455, 70 , 160, 372, 157, 567, 342, 398],
    [0  , 664, 997, 598, 507, 197, 311, 479, 310, 581, 417, 376, 275],
    [0  , 0  , 289, 744, 390, 437, 491, 645, 880, 618, 374, 455, 211],  
    [0  , 0  , 0  , 491, 265, 410, 664, 804, 1070,768, 259, 499, 310],
    [0  , 0  , 0  , 0  , 400, 514, 902, 990, 1261,947, 418, 635, 636], 
    [0  , 0  , 0  ,0   , 0  , 168, 522, 634, 910, 593, 19 , 284, 239], 
    [0  , 0  , 0  , 0  ,0  , 0   , 232, 124, 163, 439, 757, 389, 123],
    [0  , 0  , 0  , 0  ,0  , 0   , 0  , 154, 406, 133, 508, 273, 355], 
    [0  , 0  , 0  , 0  ,0  , 0   , 0  , 0  , 276, 43 , 623, 358, 498],
    [0  , 0  , 0  , 0  ,0  , 0   , 0  , 0  , 0  , 318, 898, 633, 761], 
    [0  , 0  , 0  , 0  ,0  , 0   , 0  , 0  , 0  , 0  , 582, 315, 464], 
    [0  , 0  , 0  , 0  ,0  , 0   , 0  , 0  , 0  , 0  , 0  , 275, 221], 
    [0  , 0  , 0  , 0  , 0 , 0   , 0  , 0  , 0  , 0  , 0  , 0  , 247]]   


We need to find such a vector <v| = [V1,V2,...,Vn], where n is the number of columns, of the square matrix W, which
c = <v|W|v> has a minimum value. Where Vi is a Real number [0, 1]

## calculation qubo sum

In [35]:
def calc_qubo_value(W, v):
    quboSum = 0
    for i in range(0,len(v)):
        for j in range(0, len(v)):
            quboSum += v[i]*W[i][j]*v[j]
    return quboSum


## Make random vector

In [36]:
import random
def make_random_vector(nsize):
    v = [];
    for x in range(0, nsize):
        v.append(random.random())
    return v    
    

In [37]:
vct = make_random_vector(len(W[0]))
print(vct)

[0.2479867345564598, 0.721369652454255, 0.9561687237997859, 0.34331435526189835, 0.5569300018928304, 0.5863439784833088, 0.6926191188027525, 0.6641929628097331, 0.04018661203726415, 0.6837627993058603, 0.5617769170322446, 0.1189749297075049, 0.6789212822160954]


In [38]:
qubo = calc_qubo_value(W, make_random_vector(len(W[0])))
print('qubo sum:' + str(qubo)) 

qubo sum:9248.153785155506


## randomize weight matrix 

In [39]:
#rng = np.random.default_rng()
def random_index(nsize, vindex = []):
    if len(vindex) == nsize:
        return vindex
    index = random.randint(0,nsize-1)
#    index = rng.integers(0,nsize-1)
    for i in range(0,len(vindex)):
        if vindex[i] == index: 
            return random_index(nsize, vindex)
    vindex.append(index)
    return random_index(nsize, vindex)
    

In [40]:
print(random_index(10))

[0, 3, 6, 9, 4, 5, 8, 2, 1, 7]


In [41]:
def randomize_matrix(M, rv):
    newM = [([0]*len(M[0])) for i in range(len(M[0]))]
    for i in range(0, len(rv)):
        for j in range(0, len(rv)):
            if i > j:
                newM[i][j] = 0
            else:
                if rv[i] <= rv[j]:
                    newM[i][j] = M[rv[i]][rv[j]]
                else:
                    newM[i][j] = M[rv[j]][rv[i]]
    return newM

In [42]:

rv = random_index(len(W[0]))
print(rv)
newW = randomize_matrix(W, rv)


[0, 3, 6, 9, 4, 5, 8, 2, 1, 7, 11, 10, 12]


## Matrix split

In [43]:
def split_matrix(M, size):
    M1 = []
    M2 = []
    if len(M) < size:
        return []
    for i in range(0, size):
        M1.append([])
        for j in range(0, size):
            M1[i].append(M[i][j])
    for i in range(size, len(M)):
        M2.append([])
        for j in range(size, len(M)):
            M2[i-size].append(M[i][j])
            
    return [M1, M2]


In [44]:
listM = split_matrix(W, 7)
print(listM)


[[[153, 510, 706, 966, 581, 455, 70], [0, 664, 997, 598, 507, 197, 311], [0, 0, 289, 744, 390, 437, 491], [0, 0, 0, 491, 265, 410, 664], [0, 0, 0, 0, 400, 514, 902], [0, 0, 0, 0, 0, 168, 522], [0, 0, 0, 0, 0, 0, 232]], [[154, 406, 133, 508, 273, 355], [0, 276, 43, 623, 358, 498], [0, 0, 318, 898, 633, 761], [0, 0, 0, 582, 315, 464], [0, 0, 0, 0, 275, 221], [0, 0, 0, 0, 0, 247]]]
